In [1]:
import numpy as np
from numpy.linalg import lstsq
from itertools import combinations_with_replacement
from scipy.integrate import solve_ivp
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


In [3]:
def lorenz(t, x, sigma=10.0, rho=28.0, beta=8/3):
    X, Y, Z = x
    return [sigma*(Y - X), X*(rho - Z) - Y, X*Y - beta*Z]

# simulate
dt = 0.002
T  = 20.0
t_span = (0, T)
t_eval = np.arange(0, T, dt)
x0 = [ -8.0, 8.0, 27.0 ]   # initial condition

sol = solve_ivp(lorenz, t_span, x0, t_eval=t_eval, rtol=1e-10, atol=1e-12)
t = sol.t
X = sol.y.T                    # shape (N, 3), columns = [x, y, z]

# We’ll use this only to create data; pretend that we don’t know the equations afterward.


In [4]:
# adding some noise to add robustness

noise_level = 0.0            # e.g., 0.01 for 1% noise
X_noisy = X + noise_level*np.std(X, axis=0)*np.random.randn(*X.shape)


In [5]:
X_data = X_noisy
# Use X_data = X_noisy if you add noise; otherwise X_data = X


## Estimating now the x derivatives $\dot{x}$.

SINDy assumes you have $\dot{x}$(t). With noise-free data you can finite-difference; with realistic data, a Savitzky–Golay (SG) filter is much better.

In [6]:
def estimate_derivative_savgol(X, dt, window_frac=0.05, polyorder=3):
    N = len(X)
    win = max(5, int(window_frac*N) | 1)  # odd, at least 5
    win = min(win, N-(1-N%2))             # ensure <= N and odd
    dXdt = savgol_filter(X, window_length=win, polyorder=polyorder,
                         deriv=1, delta=dt, axis=0, mode='interp')
    return dXdt

dXdt = estimate_derivative_savgol(X_data, dt)


$$ \text{Build a polynomial library } \Theta(X) $$

In [7]:
# building a polynomial library
## Include constant, linear, and quadratic terms (sufficient for Lorenz).

def monomial_exponents(d, degree):
    # all exponent tuples e with sum(e) in 1..degree
    exps = []
    for deg in range(1, degree+1):
        for idxs in combinations_with_replacement(range(d), deg):
            e = np.zeros(d, dtype=int)
            for k in idxs:
                e[k] += 1
            exps.append(tuple(e))
    return exps

def build_library(X, degree=2, include_bias=True, names=None):
    """
    X: (N,d) data matrix
    returns Theta: (N,p), feature_names: list[str], exponents: list[tuple]
    """
    N, d = X.shape
    features = []
    names_out = []
    exps = []

    if include_bias:
        features.append(np.ones((N,1)))
        names_out.append("1")
        exps.append(tuple([0]*d))

    # linear terms
    for j in range(d):
        features.append(X[:, [j]])
        names_out.append(f"x{j+1}")
        e = [0]*d; e[j]=1
        exps.append(tuple(e))

    # higher-order terms
    for e in monomial_exponents(d, degree):
        if sum(e) == 1:          # already added linear
            continue
        term = np.prod([X[:, [j]]**e[j] for j in range(d)], axis=0)
        features.append(term)
        # build a readable name
        name = []
        for j, p in enumerate(e):
            if p>0:
                name.append(f"x{j+1}^{p}" if p>1 else f"x{j+1}")
        names_out.append("".join(name).replace("x1x2","x1*x2").replace("x1x3","x1*x3").replace("x2x3","x2*x3"))
        exps.append(e)

    Theta = np.hstack(features)
    return Theta, names_out, exps

Theta_raw, feat_names, exps = build_library(X_data, degree=2, include_bias=True)
Theta_raw.shape, feat_names


((10000, 10),
 ['1', 'x1', 'x2', 'x3', 'x1^2', 'x1*x2', 'x1*x3', 'x2^2', 'x2*x3', 'x3^2'])

$$ \text{Column scaling (important for thresholding)} $$

In [10]:
# Scale each column to unit 2-norm so a single threshold λ is meaningful
def scale_columns(A):
    s = np.linalg.norm(A, axis=0)
    s[s==0] = 1.0
    return A / s, s

Theta, col_scales = scale_columns(Theta_raw)

# When you recover coefficients, you’ll unscale by dividing by col_scales

$$ \text{Sparse regression (STLSQ)} $$

In [13]:
# Sequentially threshold small coefficients to zero, then refit on the remaining terms. 
# Do this per state component.def stlsq(Theta, dXdt, lam=0.1, max_iter=20, tol=1e-8):

def stlsq(Theta, dXdt, lam=0.1, max_iter=20, tol=1e-8):
    """
    Theta: (N,p), dXdt: (N,d)
    returns Xi: (p,d) sparse coefficient matrix
    """
    N, p = Theta.shape
    _, d = dXdt.shape
    Xi, _, _, _ = lstsq(Theta, dXdt, rcond=None)

    for _ in range(max_iter):
        Xi_old = Xi.copy()
        for k in range(d):
            small = np.abs(Xi[:, k]) < lam
            Xi[small, k] = 0.0
            big_idx = ~small
            if np.any(big_idx):
                Xi[big_idx, k], _, _, _ = lstsq(Theta[:, big_idx], dXdt[:, k], rcond=None)
        if np.max(np.abs(Xi - Xi_old)) < tol:
            break
    return Xi

lam = 0.1  # tune this
Xi_scaled = stlsq(Theta, dXdt, lam=lam, max_iter=25)
# unscale coefficients back to the raw (un-normalized) library
Xi = Xi_scaled / col_scales[:, None]



In [14]:
def print_model(Xi, feat_names, var_names=("x","y","z"), tol=1e-10):
    p, d = Xi.shape
    for k, vk in enumerate(var_names, start=1):
        terms = []
        for j in range(p):
            c = Xi[j, k-1]
            if abs(c) > tol:
                terms.append(f"{c:+.5f}·{feat_names[j]}")
        rhs = " ".join(terms) if terms else "0"
        print(f"d{vk}/dt = {rhs}")

print_model(Xi, feat_names, var_names=("x","y","z"))


dx/dt = -5.56505·1 -16.97208·x1 +13.63118·x2 +1.01893·x3 +0.19516·x1^2 -0.19378·x1*x2 +0.36958·x1*x3 +0.04524·x2^2 -0.28345·x2*x3 -0.03504·x3^2
dy/dt = -0.42397·1 -7.65496·x1 +7.53936·x2 +0.54072·x3 +0.15682·x1^2 -0.13100·x1*x2 +0.07683·x1*x3 +0.01465·x2^2 -0.12469·x2*x3 -0.02413·x3^2
dz/dt = +49.59975·1 -2.17455·x1 +1.51900·x2 -5.17219·x3 +0.88247·x1^2 -0.88572·x1*x2 +0.05685·x1*x3 +0.52385·x2^2 -0.04225·x2*x3 +0.04936·x3^2


In [15]:
def coeff(feat, eq):
    j = feat_names.index(feat)
    return Xi[j, eq]

sigma_hat =  coeff("x2", 0)    # y in dx/dt
sigma_hat2 = -coeff("x1", 0)   # -x in dx/dt (should match sigma)
rho_hat   =  coeff("x1", 1)    # x in dy/dt
beta_hat  = -coeff("x3", 2)    # -z in dz/dt
xz_hat    = -coeff("x1*x3",1)  # -xz in dy/dt (should be -1)
xy_hat    =  coeff("x1*x2",2)  #  xy in dz/dt (should be +1)

print(f"sigma ≈ {sigma_hat:.4f}, {sigma_hat2:.4f} | rho ≈ {rho_hat:.4f} | beta ≈ {beta_hat:.4f}")
print(f"coeff(xz in dy) ≈ {xz_hat:.4f} | coeff(xy in dz) ≈ {xy_hat:.4f}")


sigma ≈ 13.6312, 16.9721 | rho ≈ -7.6550 | beta ≈ 5.1722
coeff(xz in dy) ≈ -0.0768 | coeff(xy in dz) ≈ -0.8857


$$ \text{sanity check against reference library} $$

In [ ]:
#!pip install pysindy
import pysindy as ps
opt = ps.STLSQ(threshold=0.1)
feat = ps.PolynomialLibrary(degree=2, include_interaction=True, include_bias=True)
model = ps.SINDy(optimizer=opt, feature_library=feat)
model.fit(X_data, t=dt)
model.print()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 8.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: zipp
    Found existing installation: zipp 3.17.0
    Uninstalling zipp-3.17.0:
      Successfully uninstalled zipp-3.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2

ValueError: solve: Input operand 1 does not have enough dimensions (has 1, gufunc core with signature (m,m),(m,n)->(m,n) requires 2)